<a href="https://colab.research.google.com/github/Nathbobs/Graduation_Capstone/blob/main/nerf_fernSample_Working_nerfStudio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
    <picture>
    <source media="(prefers-color-scheme: dark)" srcset="https://docs.nerf.studio/en/latest/_images/logo-dark.png">
    <source media="(prefers-color-scheme: light)" srcset="https://docs.nerf.studio/en/latest/_images/logo.png">
    <img alt="nerfstudio" src="https://docs.nerf.studio/en/latest/_images/logo.png" width="400">
    </picture>
</p>


# Nerfstudio: A collaboration friendly studio for NeRFs


![GitHub stars](https://img.shields.io/github/stars/nerfstudio-project/nerfstudio?color=gold&style=social)

This colab shows how to train and view NeRFs from Nerfstudio both on pre-made datasets or from your own videos/images.

\\

Credit to [NeX](https://nex-mpi.github.io/) for Google Colab format.

## Frequently Asked Questions

*  **Downloading custom data is stalling (no output):**
    * This is a bug in Colab. The data is processing, but may take a while to complete. You will know processing completed if `data/nerfstudio/custom_data/transforms.json` exists.
*   **Training is not showing progress**:
    * The lack of output is a bug in Colab. You can see the training progress from the viewer.

* **Viewer Quality is bad / Low resolution**:
    * This may be because more GPU is being used on training that rendering the viewer. Try pausing training or decreasing training utilization.

* **Other problems?**
    * Feel free to create an issue on our [GitHub repo](https://github.com/nerfstudio-project/nerfstudio).


In [1]:
#@title # Install Conda (requires runtime restart) { vertical-output: true, display-mode: "form" }

!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...


In [2]:
#@title # Install Nerfstudio and Dependencies (~10 min) { vertical-output: true, display-mode: "form" }

%cd /content/
!pip install --upgrade pip
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 -f https://download.pytorch.org/whl/torch_stable.html

# Installing TinyCuda
%cd /content/
!gdown "https://drive.google.com/u/1/uc?id=1q8fuc-Mqiev5GTBTRA5UPgCaQDzuqKqj"
!pip install tinycudann-1.6-cp37-cp37m-linux_x86_64.whl

# Installing COLMAP
%cd /content/
!conda install -c conda-forge colmap

# Install nerfstudio
%cd /content/
!pip install nerfstudio

/content
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.3.1
    Uninstalling pip-24.3.1:
      Successfully uninstalled pip-24.3.1
Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.12.1+cu113 (from versions: 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2, 2.1.0, 2.1.0+cpu, 2.1.0+cpu.cxx11.abi, 2.1.0+cu118, 2.1.0+cu121, 2.1.0+cu121.with.pypi.cudnn, 2.1.0+rocm5.5, 2.1.0+rocm5.6, 2.1.1, 2.1.1+cpu, 2.1.1+cpu.cxx11.abi, 2.1.1+cu118, 2.1.1+cu121, 2.1.1+cu121.with.pypi.cudnn, 2.1.1+roc

In [1]:
#@markdown <h1> Downloading Data</h1>
#@markdown <h3>Pick the preset scene or upload your own images/video</h3>
import os
from google.colab import files
from IPython.core.display import display, HTML

scene = '📤 upload your images' #@param ['🖼 poster', '🚜 dozer', '🌄 desolation', '📤 upload your images' , '🎥 upload your own video']
scene = ' '.join(scene.split(' ')[1:])

if scene not in ['upload your images', 'upload your own video']:
    %cd /content/
    !ns-download-data --dataset=nerfstudio --capture=$scene
else:
    display(HTML('<h3>Select your custom data</h3>'))
    display(HTML('<p/>You can select multiple images by pressing ctrl, cmd or shift and click.<p>'))
    display(HTML('<p/>Note: This may take time, especially on hires inputs, so we recommend to download dataset after creation.<p>'))
    !mkdir -p /content/data/nerfstudio/fern
    if scene == 'upload your images':
        !mkdir -p /content/data/nerfstudio/fern/fern_images
        %cd /content/data/nerfstudio/fern/fern_images
        uploaded = files.upload()
        dir = os.getcwd()
    else:
        %cd /content/data/nerfstudio/fern/
        uploaded = files.upload()
        dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    del uploaded
    %cd /content/

    if scene == 'upload your images':
        !ns-process-data images --data /content/data/nerfstudio/fern/fern_images --output-dir /content/data/nerfstudio/custom_data/
    else:
        video_path = preupload_datasets[0]
        !ns-process-data video --data $video_path --output-dir /content/data/nerfstudio/fern/

    scene = "custom_data"





/content/data/nerfstudio/fern/fern_images


Saving IMG_4026.JPG to IMG_4026.JPG
Saving IMG_4027.JPG to IMG_4027.JPG
Saving IMG_4028.JPG to IMG_4028.JPG
Saving IMG_4029.JPG to IMG_4029.JPG
Saving IMG_4030.JPG to IMG_4030.JPG
Saving IMG_4031.JPG to IMG_4031.JPG
Saving IMG_4032.JPG to IMG_4032.JPG
Saving IMG_4033.JPG to IMG_4033.JPG
Saving IMG_4034.JPG to IMG_4034.JPG
Saving IMG_4035.JPG to IMG_4035.JPG
Saving IMG_4036.JPG to IMG_4036.JPG
Saving IMG_4037.JPG to IMG_4037.JPG
Saving IMG_4038.JPG to IMG_4038.JPG
Saving IMG_4039.JPG to IMG_4039.JPG
Saving IMG_4040.JPG to IMG_4040.JPG
Saving IMG_4041.JPG to IMG_4041.JPG
Saving IMG_4042.JPG to IMG_4042.JPG
Saving IMG_4043.JPG to IMG_4043.JPG
Saving IMG_4044.JPG to IMG_4044.JPG
Saving IMG_4045.JPG to IMG_4045.JPG
/content
[03:15:00] 🎉 Done copying images with prefix 'frame_'.                                        ]8;id=540749;file:///usr/local/lib/python3.11/site-packages/nerfstudio/process_data/process_data_utils.py\process_data_utils.py]8;;\:]8;id=604694;file:///usr/local/lib/pyth

In [2]:
#@title # Set Up Viewer

%cd /content

# Install localtunnel
# We are using localtunnel https://github.com/localtunnel/localtunnel but ngrok could also be used
!npm install -g localtunnel

# Tunnel port 7007, the default for
!rm url.txt 2> /dev/null
get_ipython().system_raw('lt --port 7007 >> url.txt 2>&1 &')

/content
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 22 packages in 2s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋

In [3]:
#@title # Start Viewer

with open('url.txt') as f:
  lines = f.readlines()
websocket_url = lines[0].split(": ")[1].strip().replace("https", "wss")
# from nerfstudio.utils.io import load_from_json
# from pathlib import Path
# json_filename = "nerfstudio/nerfstudio/viewer/app/package.json"
# version = load_from_json(Path(json_filename))["version"]
url = f"https://viewer.nerf.studio/?websocket_url={websocket_url}"
print(url)
print("You may need to click Refresh Page after you start training!")
from IPython import display
display.IFrame(src=url, height=800, width="100%")

https://viewer.nerf.studio/?websocket_url=wss://olive-pants-build.loca.lt
You may need to click Refresh Page after you start training!


In [4]:
# #@title # Start Training { vertical-output: true }

# %cd /content
# !ns-train nerfacto --viewer.websocket-port 7007 nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 4

Streaming output truncated to the last 5000 lines.
                   ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyError: 'CameraMessage'
Step (% Done)       Train Iter (time)    ETA (time)           Train Rays / Sec                       
-----------------------------------------------------------------------------------                  
550 (1.83%)         362.694 ms           2 h, 58 m, 1 s       12.12 K                                
560 (1.87%)         362.655 ms           2 h, 57 m, 56 s      12.12 K                                
570 (1.90%)         354.213 ms           2 h, 53 m, 44 s      12.43 K                                
580 (1.93%)         359.021 ms           2 h, 56 m, 2 s       12.25 K                                
590 (1.97%)         362.537 ms           2 h, 57 m, 42 s      12.14 K                                
600 (2.00%)         358.302 ms           2 h, 55 m, 34 s      12.29 K                                
610 (2.03%)         354

In [8]:
#@title # Quick Training for Fast Results { vertical-output: true }

%cd /content
!ns-train nerfacto --max-num-iterations 5000 --viewer.websocket-port 7878 nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 8


Streaming output truncated to the last 5000 lines.
Step (% Done)       Train Iter (time)    ETA (time)           Train Rays / Sec                       
-----------------------------------------------------------------------------------                  
1370 (27.40%)       354.224 ms           21 m, 25 s           12.46 K                                
1380 (27.60%)       355.420 ms           21 m, 26 s           12.42 K                                
1390 (27.80%)       361.998 ms           21 m, 46 s           12.17 K                                
1400 (28.00%)       360.184 ms           21 m, 36 s           12.21 K                                
1410 (28.20%)       354.178 ms           21 m, 11 s           12.43 K                                
1420 (28.40%)       361.442 ms           21 m, 33 s           12.16 K                                
1430 (28.60%)       361.486 ms           21 m, 30 s           12.16 K                                
1440 (28.80%)       353.795 ms 

In [9]:
#@title # Render Video -- Gives Error (viewing model with nerfacto)

!ns-viewer --load-config outputs/unnamed/nerfacto/2025-05-21_033759/config.yml


/usr/local/lib/python3.11/site-packages/nerfstudio/field_components/activations.py:32: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float32)
/usr/local/lib/python3.11/site-packages/nerfstudio/field_components/activations.py:38: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/site-packages/nerfstudio/field_components/activations.py:32: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float32)
/usr/local/lib/python3.11/site-packages/nerfstudio/field_components/activations.py:38: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.


In [25]:
#@title # Render Video -- Gives Error

!ns-render camera-path --load-config /content/outputs/unnamed/nerfacto/2025-05-21_033759/config.yml --camera-path-filename camera_path.json --output-path renders/video.mp4


/usr/local/lib/python3.11/site-packages/nerfstudio/field_components/activations.py:32: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float32)
/usr/local/lib/python3.11/site-packages/nerfstudio/field_components/activations.py:38: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
Traceback (most recent call last):
  File "/usr/local/bin/ns-render", line 5, in <module>
    from nerfstudio.scripts.render import entrypoint
  File "/usr/local/lib/python3.11/site-packages/nerfstudio/scripts/render.py", line 59, in <module>
    from nerfstudio.utils.eval_utils import eval_setup
  File "/usr/local/lib/python3.11/site-packages/nerfstudio/utils/eval_utils.py", line 29, in <module>
    from nerfstudio.configs.method_configs import all_methods
  File "/usr/local/lib/python3.

In [26]:
%%writefile render_fix.py
import sys
import torch
import numpy
from nerfstudio.scripts.render import entrypoint as original_entrypoint

# Patch torch.load to use weights_only=False
original_load = torch.load
torch.load = lambda *args, **kwargs: original_load(*args, **{**kwargs, 'weights_only': False})

# Run the original entrypoint with the arguments passed to this script
if __name__ == "__main__":
    sys.exit(original_entrypoint())

Overwriting render_fix.py


In [20]:
#@title # Render Video -- My Approach after fixing torch.load to false using a script
!python render_fix.py dataset --load-config /content/outputs/unnamed/nerfacto/2025-05-21_033759/config.yml --output-path renders/dataset_video.mp4 --split train

/usr/local/lib/python3.11/site-packages/nerfstudio/field_components/activations.py:32: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float32)
/usr/local/lib/python3.11/site-packages/nerfstudio/field_components/activations.py:38: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd

🏃 🏃 Install tcnn for speedups 🏃 🏃
pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch


🏃 🏃 Install tcnn for speedups 🏃 🏃
pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch


🏃 🏃 Install tcnn for speedups 🏃 🏃
pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch


🏃 🏃 Install tcnn for speedups 🏃 🏃
pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch


🏃 🏃 Install 

In [22]:
# Installing FFmpeg
!apt-get install ffmpeg -y

#frame path
FRAMES_DIR="/content/renders/dataset_video.mp4/train/rgb"

# Creating a video with FFmpeg that handles missing frames
!ffmpeg -framerate 30 -pattern_type glob -i "$FRAMES_DIR/frame_*.jpg" -c:v libx264 -pix_fmt yuv420p -vf "setpts=N/30/TB" fern_video.mp4

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq -

In [23]:
#making a slower video
!ffmpeg -i fern_video.mp4 -filter:v "setpts=4.0*PTS" fern_video_slower.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [24]:
#@title # Render Video -- Default Approach by nerfstudio{ vertical-output: true }
#@markdown <h3>Export the camera path from within the viewer, then run this cell.</h3>
#@markdown <h5>The rendered video should be at renders/output.mp4!</h5>


# base_dir = "/content/outputs/data-nerfstudio-" + scene + "/nerfacto/"
# training_run_dir = base_dir + os.listdir(base_dir)[0]

# from IPython.core.display import display, HTML
# display(HTML('<h3>Upload the camera path JSON.</h3>'))
# %cd $training_run_dir
# uploaded = files.upload()
# uploaded_camera_path_filename = list(uploaded.keys())[0]

# config_filename = training_run_dir + "/config.yml"
# camera_path_filename = training_run_dir + "/" + uploaded_camera_path_filename
# camera_path_filename = camera_path_filename.replace(" ", "\\ ").replace("(", "\\(").replace(")", "\\)")

# %cd /content/
# !ns-render --load-config $config_filename --traj filename --camera-path-filename $camera_path_filename --output-path renders/output.mp4